## Generating binary categories for training
**Steps**

- Setup categoricals for each segment based on author consensus
- Assign a binary value to each category of segment, in corresponding category columns (one-hot encoding)

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
# from sqlalchemy import create_engine
# from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#Connect to db
dbname = 'beforeiagree_db'
username = 'peterostendorp'

#Create engine
con = psycopg2.connect(database = dbname, user = username)

In [5]:
#Get annotations df
sql = """
SELECT * FROM annotations;
"""
annotations = pd.read_sql_query(sql,con)

In [6]:
#Get sites df
sql = """
SELECT * FROM sites;
"""
sites = pd.read_sql_query(sql,con)

In [7]:
#Get segments df
sql = """
SELECT * FROM segments
"""

segments = pd.read_sql_query(sql,con)

In [8]:
annotations.head()

,Policy UID,annotation_id,batch_id,annotator_id,segment_id,category_name,attributes_value_pairs,date,policy_url
0,1017,20137,test_category_labeling_highlight_fordham_aaaaa,121,0,Other,"{""Other Type"": {""selectedText"": ""Sci-News.com ...",NaT,http://www.sci-news.com/privacy-policy.html
1,1017,20324,test_category_labeling_highlight_fordham_aaaaa,121,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",NaT,http://www.sci-news.com/privacy-policy.html
2,1017,20325,test_category_labeling_highlight_fordham_aaaaa,121,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",NaT,http://www.sci-news.com/privacy-policy.html
3,1017,20326,test_category_labeling_highlight_fordham_aaaaa,121,2,Data Retention,"{""Personal Information Type"": {""selectedText"":...",NaT,http://www.sci-news.com/privacy-policy.html
4,1017,20327,test_category_labeling_highlight_fordham_aaaaa,121,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele...",NaT,http://www.sci-news.com/privacy-policy.html


In [9]:
sites.head()

,Policy UID,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sector,Policy URL,Policy collection date,Policy last updated date,policy_text
0,20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)",Arts,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01,<strong> Privacy Policy </strong> <br> <br> <s...
1,21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Arts,imdb.com/privacy,2015-07-02,2014-12-05,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
2,26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Arts,nytimes.com/privacy,2015-07-08,2015-06-10,<strong> Privacy Policy </strong> <br> <br> La...
3,32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...
4,32,1000,voxmedia.com,Vox Media,2015-02-14,False,"Alexa Rank: 40248 (Global), 18551 (US)",None,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...


In [10]:
segments.head()

,Policy UID,segment_id,segments
0,20,0,<strong> Privacy Policy </strong> <br> <br> <s...
1,20,1,This privacy policy does not apply to Sites ma...
2,20,2,"By visiting our Sites, you are accepting the p..."
3,20,3,<strong> What Information Is Collected? </stro...
4,20,4,<strong> Personally Identifiable Information <...


In [18]:
#Set up table in which categories are assigned to each unique document segment
#Join category name to segments and determine the mode of each category, concat back onto original segments
tmp = pd.merge(annotations,segments,on=['Policy UID','segment_id'],how='outer')
tmp['category_name'] = tmp['category_name'].fillna(value='None')
tmp2 = pd.DataFrame(tmp.groupby(by=['Policy UID','segment_id'])['category_name'].agg(lambda x: x.value_counts().index[0]))

In [19]:
tmp2.head()

category_name
Policy UID segment_id                            
20         0                                Other
           1                                Other
           2                        Policy Change
           3           First Party Collection/Use
           4           First Party Collection/Use

In [22]:
categories = list(tmp2['category_name'].unique())
cols = {'Other': 'other',
        'Policy Change': 'policy_change',
        'First Party Collection/Use': 'first_party_collection_use',
        'Third Party Sharing/Collection': 'third_party_sharing_collection',
        'Do Not Track': 'do_not_track',
        'User Choice/Control': 'user_choice_control',
        'International and Specific Audiences': 'international_specific_audiences',
        'Data Security': 'data_security',
        'Data Retention': 'data_retention',
        'User Access, Edit and Deletion': 'user_access_edit_deletion'}

## Segment-level categorization
Loop through categories and generate one-hot encoding of each category and stick them into new columns with the corresponding names in cols

In [24]:
#Loop through the categories and generate a set of new columns with names in cols
binary_categories = pd.DataFrame()

for category in categories:
    one_hot = lambda s: 1 if s.startswith(category) else 0
    binary_categories[cols[category]] = tmp2['category_name'].apply(one_hot)

In [25]:
binary_categories.head()

other  policy_change  first_party_collection_use  \
Policy UID segment_id                                                     
20         0               1              0                           0   
           1               1              0                           0   
           2               0              1                           0   
           3               0              0                           1   
           4               0              0                           1   

                       third_party_sharing_collection  do_not_track  \
Policy UID segment_id                                                 
20         0                                        0             0   
           1                                        0             0   
           2                                        0             0   
           3                                        0             0   
           4                                        0             0   

                       user_choice_control  international_specific_audiences  \
Policy UID segment_id                                                          
20         0                             0                                 0   
           1                             0                                 0   
           2                             0                                 0   
           3                             0                                 0   
           4                             0                                 0   

                       data_security  data_retention  \
Policy UID segment_id                                  
20         0                       0               0   
           1                       0               0   
           2                       0               0   
           3                       0               0   
           4                       0               0   

                       user_access_edit_deletion  
Policy UID segment_id                             
20         0                                   0  
           1                                   0  
           2                                   0  
           3                                   0  
           4                                   0

In [27]:
#Create engine for persisting
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [28]:
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://peterostendorp@localhost/beforeiagree_db


In [29]:
binary_categories.to_sql('binary_segment_categories',engine, if_exists='replace')

## Policy-level categorization

In [39]:
binary_categories_policy = binary_categories.reset_index(level=[0,1]).groupby('Policy UID').sum().drop('segment_id',axis=1)
binary_categories_policy = binary_categories_policy.applymap(lambda x: 1 if x > 0 else 0)
binary_categories_policy.head()

,other,policy_change,first_party_collection_use,third_party_sharing_collection,do_not_track,user_choice_control,international_specific_audiences,data_security,data_retention,user_access_edit_deletion
Policy UID,,,,,,,,,,
20,1,1,1,1,1,1,1,1,1,1
21,1,1,1,1,0,1,1,1,0,1
26,1,1,1,1,0,1,1,1,0,1
32,1,1,1,1,0,1,1,1,1,0
33,1,1,1,1,0,1,1,1,1,1


In [40]:
binary_categories_policy.sum()

other                               112
policy_change                        83
first_party_collection_use          112
third_party_sharing_collection      110
do_not_track                         22
user_choice_control                  98
international_specific_audiences     85
data_security                        94
data_retention                       25
user_access_edit_deletion            73
dtype: int64

Most policies contain info on:
- Other
- First and third party data collection
- user choice control
- data security

Less frequently mentioned:
- data retention
- user access edit deletion
- do not track

In [41]:
binary_categories_policy.to_sql('binary_policy_categories',engine)